**Mufti Alfarokhul Azam <br> 510317**

In [389]:
import mysql.connector
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [390]:
db = mysql.connector.connect(
  host="localhost",
  user="muftialroaz",
  password="password",
  database="rekmed",
  port=3306
)

Rekam Medis

In [391]:
# Retrieve all records from the 'customers' table
cursor = db.cursor()
cursor.execute("SELECT rekam_medis.rm_id, rm_diagnosis.nama_diagnosis, dokter.user_id AS dokter_user_id, dokter.nama AS dokter_nama, user.username AS dokter_username, pasien.mr AS pasien_mr, pasien.nama AS pasien_nama FROM rekam_medis LEFT JOIN dokter ON rekam_medis.user_id = dokter.user_id LEFT JOIN rm_diagnosis ON rekam_medis.rm_id = rm_diagnosis.rm_id LEFT JOIN pasien ON rekam_medis.mr = pasien.mr LEFT JOIN user ON dokter.user_id = user.id;")

# Fetch all results as a list of tuples
rekam_medis = cursor.fetchall()
rekam_medis = pd.DataFrame(rekam_medis)
rekam_medis.rename(columns={0: 'rm_id', 1: 'diagnosis_nama', 2:"dokter_user_id", 3: 'dokter_nama', 4: "dokter_username", 5: "pasien_mr", 6:"pasien_nama"}, inplace=True)

cursor.close()
db.close()

In [392]:
from pasien import pasien_data
from diagnosis import diagnosis_data
from dokter import dokter_data
from modules import preprocess_input
import db

input_data = input()
search = preprocess_input(input_data)
nama_pasien = pasien_data(search)
diagnosis = diagnosis_data(search)
nama_dokter = dokter_data(search)
rekam_medis = db.rekam_medis.copy()

In [393]:
rekam_medis

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10
1,2,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-10
2,3,Kosong,112,Yudhistya,huroncom@gmail.com,108000003,Ahmad Surya,2016-08-10
3,4,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
4,5,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11
...,...,...,...,...,...,...,...,...
284,285,Kosong,114,dr Yudhistya SpOG,indosehat,109000071,Noah Lee,2016-11-30
285,286,Kosong,114,dr Yudhistya SpOG,indosehat,109000070,Ava Santoso,2016-11-30
286,287,Kosong,114,dr Yudhistya SpOG,indosehat,109000069,Jack Wilson,2016-11-30
287,288,Kosong,114,dr Yudhistya SpOG,indosehat,109000105,Aaliyah Davis,2016-11-30


In [394]:
# rekam_medis['cosine_diagnosis'] = diagnosis['cosine']
# rekam_medis['cosine_dokter'] = nama_dokter['cosine']
# rekam_medis['cosine_pasien'] = nama_pasien['cosine']
# rekam_medis['cosine'] = (nama_pasien['cosine'] + diagnosis['cosine'] + nama_dokter['cosine'])/3
# Menggabungkan konten dari tiga kolom ke satu kolom baru 'combined'

# rekam_medis.sort_values(by='cosine_diagnosis', ascending=False)

# diagnosis_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'diagnosis_nama']
# pasien_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'pasien_nama']
# dokter_nama = rekam_medis.loc[rekam_medis['cosine'] > 0, 'dokter_nama']

In [395]:
rekam_medis['combined_text'] = rekam_medis['pasien_nama'] + ' ' + rekam_medis['diagnosis_nama'] + ' ' + rekam_medis['dokter_nama']

In [396]:
rekam_medis

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,combined_text
0,1,influenza,112,Yudhistya,huroncom@gmail.com,108000001,Sri Wahyuni,2016-08-10,Sri Wahyuni influenza Yudhistya
1,2,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-10,Dian Kusuma Kosong Yudhistya
2,3,Kosong,112,Yudhistya,huroncom@gmail.com,108000003,Ahmad Surya,2016-08-10,Ahmad Surya Kosong Yudhistya
3,4,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11,Dian Kusuma Kosong Yudhistya
4,5,Kosong,112,Yudhistya,huroncom@gmail.com,108000002,Dian Kusuma,2016-08-11,Dian Kusuma Kosong Yudhistya
...,...,...,...,...,...,...,...,...,...
284,285,Kosong,114,dr Yudhistya SpOG,indosehat,109000071,Noah Lee,2016-11-30,Noah Lee Kosong dr Yudhistya SpOG
285,286,Kosong,114,dr Yudhistya SpOG,indosehat,109000070,Ava Santoso,2016-11-30,Ava Santoso Kosong dr Yudhistya SpOG
286,287,Kosong,114,dr Yudhistya SpOG,indosehat,109000069,Jack Wilson,2016-11-30,Jack Wilson Kosong dr Yudhistya SpOG
287,288,Kosong,114,dr Yudhistya SpOG,indosehat,109000105,Aaliyah Davis,2016-11-30,Aaliyah Davis Kosong dr Yudhistya SpOG


In [397]:
import pandas as pd
from modules import preprocess, vektorisasi, cosine_sim, bm_25

doc_pre = preprocess(rekam_medis['combined_text'])

doc_vec, search_vec = vektorisasi(doc_pre, search)
cosi = cosine_sim(doc_vec, search_vec)

  # gabungkan dalam satu dataframe
df_vector = pd.concat([rekam_medis, cosi], axis=1) 
df_vector = df_vector.sort_values(by='cosine', ascending=False)
# df_vector = df_vector[df_vector['cosine'] > 0]
df_vector.head()

,rm_id,diagnosis_nama,dokter_user_id,dokter_nama,dokter_username,pasien_mr,pasien_nama,tanggal_periksa,combined_text,cosine
144,145,Kosong,93,anita,saskyagitaamira@yahoo.com,94000009,Rendi Pratama,2016-10-27,Rendi Pratama Kosong anita,0.5
218,219,Kosong,93,anita,saskyagitaamira@yahoo.com,94000078,Siti Setiawan,2016-10-29,Siti Setiawan Kosong anita,0.5
197,198,Kosong,93,anita,saskyagitaamira@yahoo.com,94000063,Nia Permadi,2016-10-28,Nia Permadi Kosong anita,0.5
196,197,Kosong,93,anita,saskyagitaamira@yahoo.com,94000062,Arifin Setiadi,2016-10-28,Arifin Setiadi Kosong anita,0.5
195,196,Kosong,93,anita,saskyagitaamira@yahoo.com,94000061,Nita Sari,2016-10-28,Nita Sari Kosong anita,0.5


In [398]:
df_vector = df_vector[df_vector['cosine'] > 0]
def calculate_precision_recall_f1(relevance_labels, retrieved_labels):
    true_positives = sum(r and l for r, l in zip(relevance_labels, retrieved_labels))
    retrieved_positives = sum(retrieved_labels)
    actual_positives = sum(relevance_labels)

    precision = true_positives / retrieved_positives if retrieved_positives > 0 else 0
    recall = true_positives / actual_positives if actual_positives > 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1

# Example usage:
relevance_labels = [1 if score >= 0.4 else 0 for score in df_vector['cosine'].tolist()]
retrieved_labels = [1 if score > 0.0 else 0 for score in df_vector['cosine'].tolist()]

precision, recall, f1 = calculate_precision_recall_f1(relevance_labels, retrieved_labels)
print('Keyword:', input_data)
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

Keyword: anita
Precision: 0.9759, Recall: 1.0000, F1 Score: 0.9878


In [399]:
from sklearn.metrics import precision_score, recall_score, average_precision_score

# Given ground truth labels (replace with your actual labels)
ground_truth_labels = [1 if score >= 0.4 else 0 for score in df_vector['cosine'].tolist()]

# Create a binary list based on the threshold
binary_list = [1 if score > 0.0 else 0 for score in df_vector['cosine'].tolist()]

# Evaluate precision@K, recall@K, and average precision
# k_value = 10  # Replace with your desired K value
k_value = len(df_vector)
top_k_binary_list = binary_list[:k_value]
top_k_ground_truth_labels = ground_truth_labels[:k_value]

precision_at_k = precision_score(top_k_ground_truth_labels, top_k_binary_list)
recall_at_k = recall_score(top_k_ground_truth_labels, top_k_binary_list)
average_precision_at_k = average_precision_score(ground_truth_labels, binary_list)

# Print evaluation metrics
print(f"Precision@{k_value}: {precision_at_k:.4f}")
print(f"Recall@{k_value}: {recall_at_k:.4f}")
print(f"Average Precision: {average_precision_at_k:.4f}")


Precision@83: 0.9759
Recall@83: 1.0000
Average Precision: 0.9759


In [400]:
import numpy as np


cosine_similarity_scores = df_vector['cosine'].tolist()

# Given relevance criteria
relevance_labels = [1 if score >= 0.4 else 0 for score in cosine_similarity_scores]

# Total number of documents
total_documents = len(cosine_similarity_scores)

# Number of relevant documents
total_relevant_documents = sum(relevance_labels)

# K value
k_value = 10
# k_value = len(df_vector)

# Sorting documents based on cosine similarity scores
sorted_indices = np.argsort(cosine_similarity_scores)[::-1]

# Retrieving top-K documents
top_k_indices = sorted_indices[:k_value]

# Calculating precision@K and recall@K
relevant_at_k = sum(relevance_labels[i] for i in top_k_indices)
precision_at_k = relevant_at_k / k_value if k_value > 0 else 0
recall_at_k = relevant_at_k / total_relevant_documents if total_relevant_documents > 0 else 0

# Calculating average precision
avg_precision = np.mean([sum(relevance_labels[:i + 1]) / (i + 1) for i in range(k_value) if relevance_labels[i]])
print('Keyword:', input_data)
# Printing the results
print(f'Precision@{k_value}: {precision_at_k:.4f}')
print(f'Recall@{k_value}: {recall_at_k:.4f}')
print(f'Average Precision: {avg_precision:.4f}')


Keyword: anita
Precision@10: 1.0000
Recall@10: 0.1235
Average Precision: 1.0000


In [401]:
rekam_medis['combined_text'].loc[1:50]

1                          Dian Kusuma Kosong Yudhistya
2                          Ahmad Surya Kosong Yudhistya
3                          Dian Kusuma Kosong Yudhistya
4                          Dian Kusuma Kosong Yudhistya
5                          Sri Wahyuni Kosong Yudhistya
6                            Mega Utami ilang Yudhistya
7             Nita Permadi Riwayat SC dr Yudhistya SpOG
8        Nita Permadi Hamil 26 minggu dr Yudhistya SpOG
9                Arief Susanto Kosong dr Yudhistya SpOG
10             Ratna Wulandari Kosong dr Yudhistya SpOG
11                Dian Saputri Kosong dr Yudhistya SpOG
12                 Bayu Wibowo Kosong dr Yudhistya SpOG
13               Fitriani Sari Kosong dr Yudhistya SpOG
14                  Ahmad Surya Typhoid fever Yudhistya
15    Ahmad Surya Typhoid and paratyphoid fevers Yud...
16                           Lily Kim Kosong dokterjaya
17                           Lily Kim Kosong dokterjaya
18                         Sri Wahyuni Kosong Yu